<a href="https://colab.research.google.com/github/matthewbelcher/ND-HFTT/blob/main/Assignments/Assignment1/Assignment_1_MLP_Regressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1 - MLP Regression

# Student Name: Ryan Kennedy

The objective of this assignment is to make you familiar with solving **regression** problems with multi-layer perceptrons.

[starting point](https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-create-a-neural-network-for-regression-with-pytorch.md)

# Background

Neural nets estimate functions.
 - some functions have _categorical_ values (i.e., a small set of integers) that represent classes (e.g., diseased/healthy, identity (name), apple/orange/banana/grape, cat/dog). Systems that
 estimate the category from the samples (observations, measurements, patterns, etc.) are _classifiers_.
 - some fuctions are continuous (or nearly so) and learning those is called _regression_; systems that perform regression are _regressors_. Examples include invesment yield estimation, positions of vehicles viewed by a drone, age of a tree, etc.
The [universal approximation theorem](https://www.machinecurve.com/index.php/2019/07/18/can-neural-networks-approximate-mathematical-functions/)
says that any continuous map can be estimated to within a specified an error bound by a sufficiently complex network of two layers.

The steps to implement a regressor in Pytorch are very similar to those you'd follow to implement a classifier. However:
 * The input data has a different _target_ (the known value of the function for a given input, instead of a known class label for a given input).
 * The loss function will be different.
 * and, of course, the layers/activations used are problem-specific.

But the flow of data (forward and backward) is the same, and training uses the same machinery.

We'll pose a regression problem; you'll design a MLP to solve it, and you'll run experiments that can help you understand how well it worked.

The basic steps below are:
1. `import` what we need
2. Set up training data
3. Design the network
4. Configure training process: dataloader for training data, loss, optimizer, hyperparameters like number of epochs and learning rate.
5. Implement a training loop and several experiments.
6. Assess the results.


# Task 1

Import all the things you need in the cell below. A starting set of imports is provided for you.

In [ ]:
# STUDENT CODE HERE
# import all the things you need.
import os
import sys
import pickle
import random
import gdown
import tqdm.notebook as tqdm
import numpy as np
import matplotlib.pyplot as plt
import sklearn.preprocessing
import torch
import torchsummary


# Task 2

a. Use `torch.random.seed()` to seed the random number generator used by Pytorch in these experiments.

b. Determine whether CUDA is available. If so, set `device` to `torch.device('cuda')`; if not, set `device` to `torch.device('cpu')` .

In [ ]:
# STUDENT CODE GOES HERE
#



# Task 3 - Create a function

Here, you're going to create your own personal polynomial-transcendental function, depict it, and get some noisy samples from it.

In the code cell below, do these things
1. Run `np.random.seed(sid)` where `sid` is your NDID number from your ID card student ID number
1. Use `np.random.randint()` to generate three random integers between 1 and 7, inclusive, sorting them in ascending order and placing the results in a numpy array named `rands`
1. Define a function `f(x,c=rands)` which computes the scalar function $f(x) = c_0\cdot x^3 - c_1\cdot x^2 + 2\cdot c_2^2 \cdot \sin(c_3\cdot x)$ . The subscripts on the $c_i$ are the same as the array indices on the `c[i]`.

1. Create `N=512` noisy observations of $f(x)$ on the domain [-5,5] as follows:
  1. Generate a vector `X` containing `N` uniformly distributed random numbers between -5 and 5. Use `np.random.uniform()` to generate these numbers. Let the $i^{th}$ element of `X` be denoted $x_i$.
  1. Using `np.random.normal()`, generate a vector `Z` of `N` samples from a normal (Gaussian) distribution with mean (location) 0 and standard deviation (scale) 10. Let the $i^{th}$ element of `Z` be denoted $z_i$.
  2. Generate a vector `yo`, where the $i^{th}$ element $yo_i = f(x_i) + z_i$.
1. Plot the function $f(x)$ on the domain [-5, 5] as a black line and plot the samples $(x_i,yo_i)$ as blue dots on the same plot.  Hints:
 + `np.linspace()` can give you a set of equally-spaced $x$ values that you can feed to $f()$ to get the corresponding $y$ values for the black line plot.
 + If you use matplotlib, `plt.plot()` is the sensible way to do the line plot, followed by `plt.scatter()` for the dots, followed by `plt.slabel()` and `plt.ylabel()` for axis labels "$x$" and "$f(x)$", followed by `plt.show()` to render the whole thing.

In [ ]:
# STUDENT CODE GOES HERE
np.random.seed( ... )

rands = sorted(list(np.random.randint( ... )))

def f(x,c=rands):
    return ...

# sample the function, noisily, at random places in the domain [-5,5]
nSamples = ...
X = np.random.uniform( ... )
Z = np.random.normal( ... )
yo = ...

xplot = np.linspace(start=-5, stop=5, num=1000)
yplot = ...

# plotting things ...



# Task 2: data set

Write code that implements a subclass of `torch.utils.data.Dataset` named `FuncDataset`, with `__init__(self,X,y)`, `__len__(self)`, and `__getitem__(self,idx)` methods that instantiate an instance with input and output samples, provide its length, and provide the `idx`'th (X,y) tuple from the instance.

You should `np.vstack()` `X` and `y` before storing them in instance variables. Torch really wants individual samples and targets to be rows in an array (tensor) and numpy doesn't do that by default.

Remember - it's not necessary to call the superclass constructor in the `__init__()` method, although it does not harm anything if you do.

In [ ]:
# STUDENT CODE GOES HERE
class FuncDataset(torch.utils.data.Dataset):
    def __init__(self,X,y):
        """ initialize the object with numpy arrays of X and y data """
        ...


    def __len__(self):
        ...

    def __getitem__(self,idx):
        ...


# Task 3: Instantiate data set; create dataloaders

Instantiate `FuncDataSet` with `x` and `yo` (calculated in Task 1) as the data and target values.

Use `torch.utils.data.random_split()` with `f_ds` and a 75%:25% training/testing split to create two new datasets named `train_data` and `test_data`; wrap these in `torch.utils.data.DataLoader`s with a batch size of 64 and `shuffle=True`.

In [ ]:
# STUDENT CODE GOES HERE

# instantiate data set

# define fractions and split the data set

# define batch_size and attach dataloaders to the datasets


# Task 4: Define MLP network

Write code that implements a subclass of `torch.nn.Module` named `FMLP`. It has one `Linear` input layer, two `Linear` hidden layers of `nHidden` units each, and a `Linear` output layer of one unit. Thus, it consumes scalar input, does _neural network magic_ with it, and provides a scalar output.  Use `torch.nn.ReLU()` activations.

I recommend wrapping the layers in `torch.nn.Sequential` in the constructor because it's cleaner, and makes the `forward()` method short.

In [ ]:
#STUDENT CODE GOES HERE
class FMLP(torch.nn.Module):
    """function approximation MLP."""
    def __init__(self,nHidden=10):
        """constructor."""
        super(FMLP,self).__init__()
        ...

    def forward(self,x):
        """predict f(x)."""
        ...

# Task 5: Experiment Function

Here, you'll define a function that runs a single experiment with a hyperparameter (the number of hidden layer units) that can be conveniently adjusted. The function returns the training loss, testing loss, and the trained model.

Implement a training loop function `fmlptrain(nHidden=1)` that does this:
1. instantiate an FMLP with the specified number of hidden units
2. choose a loss function (use whichever of `MSELoss()` or `L1Loss()` that you didn't use in the other notebook for this assignment)
3. Choose an optimizer (`Adam` or `SGD` are fine choices)
4. run a training loop for 1536 (= 1024 + 512) epochs
5. Print out the running training loss divided by the length of the training dataloader at the end of every 128th epoch.
6. Also print out the training loss at the end of training.
7. Calculates the loss on the testing data.
7. returns a tuple:`(final training loss/len(train_loader), testing loss/len(test_loader), model.to('cpu'))`

Note: It's wise to make sure the trained model is on the CPU, not the GPU at end-of-function time; hence the `.to('cpu')`

In [ ]:
#STUDENT CODE GOES HERE
def fmlptrain(nHidden=1):
    # instantiate net

    # define loss fn, LR, optimizer, max_epochs

    # move net to device, put in training mode


    for epoch in tqdm.trange(max_epochs):
        # initialize curent loss
        # loop over batches in train_loader
            # convert X, y to float and move to device
            # zero out gradient estimates in the optimizer
            # predict
            #compute loss
            # backpropagate
            # step optimizer
            # accumulate loss
        # if epoch number mod 128 == 0, print current training loss/len(train_loader)
    # calculate testing loss (make sure net is in eval mode)
    # return training loss/len(train_loader), testing loss/len(test_loader), trained model.to('cpu')


# Task 6: Experiment Execution

Run the `fmlptrain()` function for the following 14 values of `nHidden`: `[1, 2, 3, 4, 6, 8, 12, 16, 24, 32, 48, 64, 96, 128]`. Save the network after you've trained it. Make a plot of the final training loss in green and the test loss in red versus `nHidden` - use a log scale for the `x` axis of the graph to avoid squashing all the small-x values together.

To save a trained model, you have to yank it out of the GPU (if it's there) and make sure it's in evaluation mode. If you store the networks in a dict that is keyed by the value of nHidden, this will work: `model_dict[nHidden] = fmlp.to('cpu').eval()`

In [ ]:
# STUDENT CODE GOES HERE

# initialize dicts for losses, models
# loop over all values of nHidden in the experiment
    # run training, get trainloss, testloss, model
    # store losses and model in the dicts

# plot training losses versus nHidden in red (use 'training' label; legend will use it)
# plot testing lodded versus nHidden in green (use 'testing' label)
# set x axis to log scale (plt.xscale())
# set axis labels
# create a legend
# plt.show()

# Task 7: Visualize and Comment

Write code that renders (as a line plot) the original function $f(x)$, and the approximation learned by each of the 14 models trained above. Each of these line plots must be a different color (use `label=` in `plt.plot()` to do this the easy way). The basic approach is something like:
 + If necessary, use `linspace()` to generate 500 values of x between -5 and 5. Call that `xplot`
 + `plt.plot(xplot,f(xplot), ...)`
 + prep xplot for presentation to trained nets: `X = torch.tensor(np.vstack(xplot)).float()`
 + foreach value of `nHidden`:
   + let `m` be the model trained with `nHidden` hidden units
   + `y = m(X).detach().numpy()`
   + `plt.plot(xplot,y, ...)`
 + `plt.legend()`
 + `plt.show()`

 Comment in the area immediately below about the quality of the approximation as a function of the number of hidden units. Was this expected?  If you want to disable some plots and rerun the cell to look at a subset of curves with less clutter, please feel free to do so.


 # Answer:
Surprising how few hidden layer units are needed. Different students may have different experiences.

In [ ]:
# STUDENT CODE GOES HERE

# create tensor from vstracked x-values from linspace, and .float() the result, call it X
# for each value of nHidden
    # get the model for nHidden
    # present X to the model, get y, .detach().numpy() it, call it yplot
    # plot yplot vs x-values from linspace, specify nHidden as label
# plot original function values f(x) for linspace x-values, label it 'f(x)'
# create a legend
# plt.show()
